Pagenation and cursor objects for accessing specific tweets of certain users and things of the like. 
- [about tweepy's cursor.]((http://docs.tweepy.org/en/3.7.0/cursor_tutorial.html)) 
- [Cursoring explained at Twitter Developer website. ](https://developer.twitter.com/en/docs/basics/cursoring.html)



As usual, i will be building upon the code of the previous day, day#2. 
For today, there are two more imports to do: 

In [ ]:
from tweepy import API
from tweepy import Cursor

For the authentication part, it can be separated in its own class, instead of doing it inside the TwitterStreamer() class. 


In [ ]:
class TwitterAuthenticator():
    def  authenticate_twitter(self):
        auth = OAuthHandler(twitter_credentials.consumer_key, twitter_credentials.consumer_secret)
        auth.set_access_token(twitter_credentials.access_token, twitter_credentials.access_token_secret)   
        return auth


and then instatiate an object of the authentication class in the constructor of TwitterStreamer class, and then use that object where we had the previous authentication code. 

In [ ]:
class TwitterStreamer():
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()##

    def stream_tweets(self, tweets_filename, hashtags_list):
        listener = MyListener(tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter()##
        stream = Stream(auth, listener)
        stream.filter(track=hashtags_list)

Another thing that worth mentioning is the *rate limit* imposed by twitter API that locks you out from accessing information. It happened when you exceed the number of access requests within a timely window. 


More information and tips to avoid being rate limited [are available here. ](https://developer.twitter.com/en/docs/basics/rate-limiting.html)

One way to avoid being locked out because of rate_limit error, is returning false when the first rate_limit warning occurs. 
We can do that by tweaking the on_error method we defined earlier in myListener() class that inherits from StreamListener. 


In [ ]:
def on_error(self, status):
        if status ==420:
            return False
        print(status)

![](https://i.imgur.com/SPPXFF1.jpg)

![](https://i.imgur.com/UK9y9bn.jpg)

![](https://i.imgur.com/qFLL29y.jpg)

![](https://i.imgur.com/mkNvUHU.jpg)

Now comes the part of making use of the Cursor module that we imported, in retreiving tweets based on a specified user-name for example. 
- We need here to create a class, let's call it TwitterClient, and then use the Cursor to iterate through all the tweets of a given user. 
- Instead of specifying the username we want to extract tweets from, and the number of extracted tweets, in the main part of the code, we can do this in the TwitterClient class using Cursor, and then: 
- instantiating an object from the TwitterClient in the main part. 

In [ ]:
class TwitterClient():
    def __init__(self):
        self.auth=TwitterAuthenticator.authenticate_twitter()
        self.twitter_client = API(self.auth)
        
    def get_user_tweets(self, num_tweets):
        tweets = []
        for t in Cursor(self.twitter_client.user_timeline).items(num_tweets):
            tweets.append(t)
        return tweets

![](https://i.imgur.com/ftCPY2f.jpg)

The edited main part of the code will look something like this: 

In [ ]:
if __name__ == '__main__':
 
    #hash_tag_list = ["#تسقط_بس", "#مدن_السودان_تنتفض", "#SudanUprise", "#SudanRevolts"]
    #fetched_tweets_filename = "tweets.txt"

    twitter_client = TwitterClient()
    print(twitter_client.get_user_tweets(1))
    
   # twitter_streamer = TwitterStreamer()
   # twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

This way, I have used the Cursor to retreive the last n tweets.
I have not specified a user, so the default will be my own twitter-user that i used when registering the app. 


If I want to specify a username from which I want the extraction to happen, some edits are to be made in the recently newly created class TwitterClient, 
- passing a variable to the TwitterClient constructor and giving it the name of the twitter-user, or "None" for default. 
- we tell the Cursor about that, using the id. 
- when you do the instantiation of TwitterClient class in the main part of the code, dont forget to pass the username as an argument. 


In [ ]:
class TwitterClient():
    def __init__(self, twitter_user=None): ##
        self.auth=TwitterAuthenticator().authenticate_twitter()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
            
    def get_user_tweets(self, num_tweets):
        tweets = []
        for t in Cursor(self.twitter_client.user_timeline, id= self.twitter_user).items(num_tweets):##
            tweets.append(t)
        return tweets


To-read: 
- [API Reference of tweepy](http://docs.tweepy.org/en/3.7.0/api.html)